## Knowledge tree

* 负载均衡
    * DNS负载均衡
    * 算法
        * 无状态链接
        * 有状态链接 - 一致性哈希
    * 策略
        * 简单轮询
        * 最闲轮询
        * 加权轮询
* 数据一致性
    * paxos算法
* 服务可用性
* 过载
* 协议
    * TCP
    * HTTP
    * SOAP
* 编码与序列化
    * xml/json
    * parquet
* 架构框架
    * 垂直架构
    * RPC
    * SOA
* 分布式锁
    * Chubby
    * Zookeeper
    
* 模式
    * 发布订阅
    * master/slave
    * leader/follower
    * active/standby

#### 系统架构演化
* 垂直架构：MVC，一个大而全的项目，放在一个服务器里
* RPC：远程过程调用，把不同的部分，业务逻辑，数据存储拆分到不同的服务器，通过网络远程协议调用
    * 缺点：这些过程没有中心化管理，很难监控和管理
        * 协议栈
* SOA: RPC + 服务治理中心
* 微服务： 在SOA的基础上，又把服务器虚拟化，拆分到一个功能一个服务。e.g 利用docker

##### 问题
* 这些架构都没有提供高可用性，服务器挂了怎么办？
* 提供高可用性，那么意味着一个服务有多个节点提供，怎么调度？怎么把服务挂起？在服务中路由？
    * solution：
        * 集群
        * 负载均衡/反向代理(proxy)： nginx
        * 调度算法：random robin，轮询，hash-ip, ...
        
##### 负载均衡
区别是client知不知道地址

* 正向代理：知道你要去哪儿 vpn， google.com -> go
* 反向代理：到了一个中心，不用关心去哪儿，我说要什么服务，中心替我找
* session, cookie: 两个节点中，session不能共享，很多人喜欢在session中存数据，e.g 登陆验证。在节点中切换中，登陆不停失效
    * solution
        * A.不要让一个客户的请求在不同的节点中跳转, 依靠调度算法
        * B.session共享，session在集群中同步，不推荐, 因为session会占用链接，而每个服务器的请求的数目是有限制的，session共享严重限制了可以处理的请求数。 共享数据要占用一个链接，而且同步还消耗性能，性能反而遍地
    * 牵扯到调度算法
        * 轮循算法：每个请求就不听跳转 + redis 可以解决
        * hash-ip：

In [1]:
def test_china_bond_business_announcement(header, bond_type, action_type):
    rule = RULES[('si136', 'ChinaBond_BusinessAnnouncements_BOT')]
    classifier = Manager._build_classifier(rule)
    _action_type, _bond_type = classify.classify(header, classifier)
    assert bond_type == _bond_type
    assert action_type == _action_type

In [2]:
def test_ppn(header, bond_type, action_type):
    rule = RULES[('gd3087', 'ChinaMoney_PPN_BOT')]
    classifier = Manager._build_classifier(rule)
    _action_type, _bond_type = classify.classify(header, classifier)
    assert bond_type == _bond_type
    assert action_type == _action_type

##### SOA
* RPC + 服务治理中心
* 利用zookeeper来提供治理

###### Zookeeper
* 调度中心
* clients 告诉 zookeeper需要什么服务，自己的地址
* zookeeper 查询服务，告诉client， cients自己去找service
* 阿里提供dubbo 提供SOA服务治理框架

### 网络 通信框架
##### 分类
* 长链接
* 短链接


* blocking io/BIO
* Not blocking io/NIO
* Async IO/AIO

##### 可靠性保证
* 链路有效性检测
    * tcp层面的heartbeat检测
    * 协议层的heartbeat检测，e.g SMPP proxy
    * 应用层的heartbeat检测
* 断连重连机制
    * 链路失败时，一定要释放资源 handler（句柄）
    * client等待一段时间后尝试重新连接，（expenencial backoff）
* 消息缓存重发，queue

##### 协议

##### 序列化
* parquet
* Avro
* Thrift
* PB
* MessagePack


##### 锁
* 悲观锁
* 乐观锁
* 死/活锁
* 读/写锁

##### zookeeper + dubbo
https://www.bilibili.com/video/av27237493/?p=4

###### 分布式引发的问题
* 分开到不同的服务器 -> 网络通信
* 怎么找到需要的服务器地址，**动态寻址**，通信 => 服务的注册和发现

#### 项目 -  秒杀系统
https://www.bilibili.com/video/av52467230?from=search&seid=9841064693621071365

###### 业务逻辑
* 商品查询 -> 创建订单 -> 订单支付 -> 卖家发货
* 创建库存
    * 加入购物车 -> 确认订单 -> 修改库存 -> 待支付
    
* 难点：修改库存，上千上万人争几个库存

###### 技术难点
* 短时间，高并发，负载压力大
* 读多写少
* 竞争资源有限的，不能多

